In [1]:
import pandas as pd
import sqlite3 as lite
import ipywidgets as widgets
from ipywidgets import Layout, Box, HBox, VBox, interact
from IPython.display import Javascript, display, clear_output
from tkinter import *
from tkinter.filedialog import askopenfilename
import sys
import time
import threading
import os
#from automation import TaskManager, CommandSequence
#from urllib.request import Request, urlopen
#from __future__ import absolute_import
#from six.moves import range

# Detección de Fingerprinting en sitios web

##  1. Ingrese la lista de sitios

Indique un conjunto de sitios web a explorar por OpenWPM. Existen dos opciones:
* **CSV**: Permite la carga de un fichero CSV con la presentación indicada.
* **Manual**: Presenta un recuadro en el que es posible escribir las distintas URL de los sitios web. 

In [2]:
def run_cell(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

bupload = widgets.Button(button_style='success', description="Cargar")
bupload.on_click(run_cell)

txtArea = widgets.Textarea(rows=10)
buttonm = widgets.ToggleButton(
    value=False,
    description='Confirmar',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Confirmar',
    icon='check')

tab1 = HBox(children=[bupload, widgets.HTML(value='<p style="padding-left:50px"> Presione el botón <b>Cargar</b> para buscar un fichero CSV en su disco.<br> El contenido del fichero CSV debe presentar el siguiente patrón:</p><ul style="padding-left:70px"><li> 1,sitio1.com</li> <li>2,sitio2.net</li> <li>3,sitio3.org</li></ul> <p style="padding-left:50px">Nota: Si ha ingresado sitios web empleando la opción "Manual", asegúrese que el botón "Confirmar lista" no esté activado</p>')])
tab2 = HBox(children=[txtArea, buttonm, widgets.HTML(value='<p style="padding-left:50px"> Ingrese las URL a rastrear en el recuadro de la izquierda.<br> Instrucciones:</p> <ul style="padding-left:70px"><li>Incluya el protocolo (http, https) en cada dirección.</li><li>Separe cada elemento con una coma.</li>Por ejemplo:<br> https://www.sitio1.com, https://www.sitio2.net, http://www.sitio3.org </ul><ul style="padding-left:70px"><li>Presione el botón <b>Confirmar</b> para activar el uso del conjunto ingresado.</li></ul>')])

tab = widgets.Tab(children=[tab1, tab2])
tab.set_title(0, 'CSV')
tab.set_title(1, 'Manual')
VBox(children=[tab])

VBox(children=(Tab(children=(HBox(children=(Button(button_style='success', description='Cargar', style=ButtonStyle()), HTML(value='<p style="padding-left:50px"> Presione el botón <b>Cargar</b> para buscar un fichero CSV en su disco.<br> El contenido del fichero CSV debe presentar el siguiente patrón:</p><ul style="padding-left:70px"><li> 1,sitio1.com</li> <li>2,sitio2.net</li> <li>3,sitio3.org</li></ul> <p style="padding-left:50px">Nota: Si ha ingresado sitios web empleando la opción "Manual", asegúrese que el botón "Confirmar lista" no esté activado</p>'))), HBox(children=(Textarea(value='', rows=10), ToggleButton(value=False, button_style='success', description='Confirmar', icon='check', tooltip='Confirmar'), HTML(value='<p style="padding-left:50px"> Ingrese las URL a rastrear en el recuadro de la izquierda.<br> Instrucciones:</p> <ul style="padding-left:70px"><li>Incluya el protocolo (http, https) en cada dirección.</li><li>Separe cada elemento con una coma.</li>Por ejemplo:<br> https://www.sitio1.com, https://www.sitio2.net, http://www.sitio3.org </ul><ul style="padding-left:70px"><li>Presione el botón <b>Confirmar</b> para activar el uso del conjunto ingresado.</li></ul>')))), _titles={'0': 'CSV', '1': 'Manual'}),))

In [ ]:
root = Tk()
#root.update()
flName = askopenfilename(initialdir = "/",title = "Selecciona el fichero",filetypes = [("CSV - Comma-separated values","*.csv")])
root.destroy()
if not flName:
    print("Se ha cancelado la acción. Por favor, cargue un fichero CSV.")
else:
    print ("El siguiente fichero se ha cargado correctamente: " + os.path.split(flName)[1])

##  2. Rastrear sitios web

Presione el botón **Ejecutar OpenWPM** para iniciar el rastreador web con la lista de sitios ingresada.

Cuando la ejecución de la herramienta finalice, debe observar el siguiente mensaje: **Ejecución finalizada**.

In [3]:
button = widgets.Button(button_style='info',description="Ejecutar OpenWPM")
button.on_click(run_cell)
display(button)

Button(button_style='info', description='Ejecutar OpenWPM', style=ButtonStyle())

In [ ]:
if buttonm.value is True:
# The list of sites that we wish to crawl
    NUM_BROWSERS = 3
    sites = txtArea.value.split(',')

# Loads the manager preference and 3 copies of the default browser dictionaries
    manager_params, browser_params = TaskManager.load_default_params(NUM_BROWSERS)

# Update browser configuration (use this for per-browser settings)
    for i in range(NUM_BROWSERS):
    # Record HTTP Requests and Responses
        browser_params[i]['http_instrument'] = True
    # Enable flash for all three browsers
        browser_params[i]['disable_flash'] = False
    browser_params[0]['headless'] = True  # Launch only browser 0 headless

# Update TaskManager configuration (use this for crawl-wide settings)
    manager_params['data_directory'] = os.getcwd()
    manager_params['log_directory'] = os.getcwd()

# Instantiates the measurement platform
# Commands time out by default after 60 seconds
    manager = TaskManager.TaskManager(manager_params, browser_params)

# Visits the sites with all browsers simultaneously
    for site in sites:
        command_sequence = CommandSequence.CommandSequence(site)

    # Start by visiting the page
        command_sequence.get(sleep=0, timeout=60)

    # dump_profile_cookies/dump_flash_cookies closes the current tab.
        command_sequence.dump_profile_cookies(120)

    # index='**' synchronizes visits between the three browsers
        manager.execute_command_sequence(command_sequence, index='**')

# Shuts down the browsers and waits for the data to finish logging
    manager.close()


elif buttonm.value is False:
# Constants
    NUM_BROWSERS = 3
    output_dir = os.getcwd()

# Input sites list
    site_list = flName

    def get_site(line):
        return 'http://' + line.strip().split(',')[1] if line.count(',') >= 1 else None

    start_site_index = 0

# Loads the manager preference and the default browser dictionaries
    manager_params, browser_params = TaskManager.load_default_params(NUM_BROWSERS)

# Update browser configuration (use this for per-browser settings)
    for i in range(NUM_BROWSERS):
    # Record HTTP Requests and Responses
        browser_params[i]['http_instrument'] = True
    # Enable flash for all three browsers
        browser_params[i]['disable_flash'] = False
    browser_params[0]['headless'] = True  # Launch only browser 0 headless

# Update TaskManager configuration (use this for crawl-wide settings)
    manager_params['data_directory'] = output_dir
    manager_params['log_directory'] = output_dir

# Instantiates the measurement platform
    manager = TaskManager.TaskManager(manager_params, browser_params)

# Visits the sites


    def crawl_site(site, manager):
        command_sequence = CommandSequence.CommandSequence(site, reset=True)
    # Start by visiting the page
        command_sequence.get(sleep=0, timeout=60)
    # dump_profile_cookies/dump_flash_cookies closes the current tab
        command_sequence.dump_profile_cookies(120)
    # index='**' synchronizes visits between the three browsers
        manager.execute_command_sequence(command_sequence, index='**')


# Read site list
    index = 0
    with open(site_list) as f:
        for line in f:
            index += 1
            if index < start_site_index:
                continue
            site = get_site(line)
            if site is not None:
                crawl_site(site, manager)

# Shuts down the browsers and waits for the data to finish logging
    manager.close()

print("Ejecución finalizada")

## 3. Resultados

Para obtener la lista de los sitios que aplican una técnica de *fingerprinting* en particular, presione el botón que corresponda con la técnica de su interés.

In [4]:
button = widgets.Button(button_style='info',description="Objetos Informativos")
button.on_click(run_cell)
display(button)

Button(button_style='info', description='Objetos Informativos', style=ButtonStyle())

In [ ]:
if os.path.isfile("crawl-data.sqlite"):
    
    print("Procesando...")
    clear_output(wait=True)

# Conecta con la base de datos que origina OpenWPM como resultado
    wpm_db = "crawl-data.sqlite"
    conn = lite.connect(wpm_db)
    cur = conn.cursor()

# Define los valores que permiten comprobar las condiciones de fingerprinting por objetos informativos de JS
    info_ob= ["window.navigator.appCodeName", "window.navigator.appName", "window.navigator.appVersion", "window.navigator.buildID", "window.navigator.cookieEnabled", "window.navigator.doNotTrack", "window.navigator.geolocation", "window.navigator.language", "window.navigator.languages", "window.navigator.onLine", "window.navigator.oscpu", "window.navigator.platform", "window.navigator.product", "window.navigator.productSub", "window.navigator.userAgent", "window.navigator.vendorSub", "window.navigator.vendor", "window.screen.pixelDepth", "window.screen.colorDepth"]
    navplug= "window.navigator.plugins"
    navmim= "window.navigator.mimeTypes"
    ver_info_1 = set()
    ver_info_2 = set()
    info_sites_1 = set()
    info_sites_2 = set()
    info_sites = set()

# Busca en la base de datos, verificando los sitios donde se cumplen las dos condiciones fingerprinting por objetos informativos de JS

    for url, symbol, op, val, arg, top_url in cur.execute("SELECT distinct j.script_url, j.symbol, j.operation, j.value, j.arguments, v.site_url FROM javascript as j JOIN site_visits as v ON j.visit_id = v.visit_id WHERE j.symbol LIKE '%window%' ORDER BY v.site_url;"):
    
        if navplug in symbol:
            ver_info_1.add(url + symbol)
            if (sum((url + symbol) in s for s in ver_info_1)) > 5:
                info_sites_1.add(top_url + ' ' + url)
        elif navmim in symbol:
            ver_info_1.add(url + symbol)
            if (sum((url + symbol) in s for s in ver_info_1)) > 3:
                info_sites_1.add(top_url + ' ' + url)

        if symbol in info_ob:
            ver_info_2.add(url + symbol)
            if (sum((url) in s for s in ver_info_2)) > 15:
                info_sites_2.add(top_url + ' ' + url)

# Agrupa los sitios que cumplen las condiciones del fingerprinting por objetos informativos de JS (Interseccion en un solo set)
    info_sites = info_sites_1 | info_sites_2

    if len(info_sites) == 0:
        print('Ninguno de los sitios evaluados aplica esta técnica de fingerprinting')

    else:
# Separamos el set en dos listas
        a, b = zip(*(s.split(' ') for s in info_sites))

# Creamos un dataframe con las dos listas
        df1 = pd.DataFrame({'URL Sitio Web': a, 'URL Script': b})
        df1 = df1[['URL Sitio Web', 'URL Script']]

        def make_clickable(val):
            return '<a href="{}">{}</a>'.format(val,val)

#pd.options.display.max_colwidth
        pd.set_option('max_colwidth', -1)

        print('Los sitios web que aplican Fingerprinting por objetos informativos de JavaScript según los criterios analizados son los siguientes:')
        display(df1.style.format(make_clickable))

else:
    print('No se ha podido encontrar la base de datos resultante de ejecutar OpenWPM')

In [5]:
button = widgets.Button(button_style='info',description="Canvas")
button.on_click(run_cell)
display(button)

Button(button_style='info', description='Canvas', style=ButtonStyle())

In [ ]:
if os.path.isfile("crawl-data.sqlite"):
    
    print("Procesando...")
    clear_output(wait=True)

# Conecta con la base de datos que origina OpenWPM como resultado
    wpm_db = "crawl-data.sqlite"
    conn = lite.connect(wpm_db)
    cur = conn.cursor()

# Define los valores que permiten comprobar las condiciones de canvas fingerprinting
    canvas_h = "HTMLCanvasElement.height"
    canvas_w = "HTMLCanvasElement.width"
    canvas_cf= "CanvasRenderingContext2D.fillStyle"
    canvas_cs= "CanvasRenderingContext2D.strokeStyle"
    canvas_ff= "CanvasRenderingContext2D.fillText"
    canvas_fs= "CanvasRenderingContext2D.strokeText"
    canvas_save= "CanvasRenderingContext2D.save"
    canvas_rest= "CanvasRenderingContext2D.restore"
    canvas_el= "HTMLCanvasElement.addEventListener"
    canvas_data= "HTMLCanvasElement.toDataURL"
    canvas_img= "CanvasRenderingContext2D.getImageData"
    fp_1 = set()
    fp_2 = set()
    fp_3 = set()
    fp_4 = set()
    veri_1 = set()
    veri_2 = set()
    fp_sites = set()

# Busca en la base de datos, verificando los sitios donde se cumplen las cuatro condiciones de canvas fingerprinting

    for url, symbol, op, val, arg, top_url in cur.execute("SELECT distinct j.script_url, j.symbol, j.operation, j.value, j.arguments, v.site_url FROM javascript as j JOIN site_visits as v ON j.visit_id = v.visit_id WHERE j.symbol LIKE '%Canvas%' ORDER BY v.site_url;"):

        if canvas_h in symbol: 
            if val != "null" and float(val) >= 16:
                veri_1.add(symbol + url)
                if (canvas_w + url) in veri_1:
                    fp_1.add(top_url + ' ' + url)
        elif canvas_w in symbol:
            if val != "null" and float(val) >= 16:
                veri_1.add(symbol + url)
                if (canvas_h + url) in veri_1:
                    fp_1.add(top_url + ' ' + url)
            
        if canvas_cf in symbol:
            veri_2.add(symbol + url + val)
            if (canvas_cs + url + val) in veri_2:
                fp_2.add(top_url + ' ' + url)
            elif (sum((canvas_cf + url) in s for s in veri_2)) > 1:
                fp_2.add(top_url + ' ' + url)
        elif canvas_cs in symbol:
            veri_2.add(symbol + url + val)
            if (canvas_cf + url + val) in veri_2:
                fp_2.add(top_url + ' ' + url)
            elif (sum((canvas_cs + url) in s for s in veri_2)) > 1:
                fp_2.add(top_url + ' ' + url)
        elif canvas_ff in symbol and (len(set(arg[arg.find('0":"')+4:arg.find(',"1"')-1])))>= 10:	
            fp_2.add(top_url + ' ' + url)
        elif canvas_fs in symbol and (len(set(arg[arg.find('0":"')+4:arg.find(',"1"')-1])))>= 10:	
            fp_2.add(top_url + ' ' + url)

        if canvas_save in symbol:
            fp_3.add(top_url + ' ' + url)
        elif canvas_rest in symbol:
            fp_3.add(top_url + ' ' + url)
        elif canvas_el in symbol:
            fp_3.add(top_url + ' ' + url)

        if canvas_data in symbol:
            fp_4.add(top_url + ' ' + url)
        elif canvas_img in symbol:
            if float(arg[arg.find('"2":')+4:arg.find(',"3"')]) >= 16:
                if float(arg[arg.find('"3":')+4:arg.find('}')]) >= 16:
                    fp_4.add(top_url + ' ' + url)

# Agrupa los sitios que cumplen las cuatro condiciones del canvas fingerprinting (Interseccion en un solo set)
    fp_sites = (fp_1 & fp_2 & fp_4) - fp_3
        
    if len(fp_sites) == 0:
        print('Ninguno de los sitios evaluados aplica esta técnica de fingerprinting')

    else:
# Separamos el set en dos listas
        c, d = zip(*(s.split(' ') for s in fp_sites))

# Creamos un dataframe con las dos listas
        df2 = pd.DataFrame({'URL Sitio Web': c, 'URL Script': d})
        df2 = df2[['URL Sitio Web', 'URL Script']]

        def make_clickable(val):
            return '<a href="{}">{}</a>'.format(val,val)

#pd.options.display.max_colwidth
        pd.set_option('max_colwidth', -1)

        print('Los sitios web que aplican Canvas Fingerprinting según los criterios analizados son los siguientes:')
        display(df2.style.format(make_clickable))

else:
    print('No se ha podido encontrar la base de datos resultante de ejecutar OpenWPM')

In [6]:
button = widgets.Button(button_style='info',description="Canvas-Fuentes")
button.on_click(run_cell)
display(button)

Button(button_style='info', description='Canvas-Fuentes', style=ButtonStyle())

In [ ]:
if os.path.isfile("crawl-data.sqlite"):
    
    print("Procesando...")
    clear_output(wait=True)

# Conecta con la base de datos que origina OpenWPM como resultado
    wpm_db = "crawl-data.sqlite"
    conn = lite.connect(wpm_db)
    cur = conn.cursor()

# Define los valores que permiten comprobar las condiciones de canvas font fingerprinting
    canvas_font= "CanvasRenderingContext2D.font"
    canvas_text= "CanvasRenderingContext2D.measureText"
    fontp_1 = set()
    fontp_2 = set()
    verif_1 = set()
    verif_2 = list()
    fontp_sites = set()

# Busca en la base de datos, verificando los sitios donde se cumplen las dos condiciones de canvas font fingerprinting

    for url, vid, symbol, op, val, arg, top_url in cur.execute("SELECT j.script_url, j.visit_id, j.symbol, j.operation, j.value, j.arguments, v.site_url FROM javascript as j JOIN site_visits as v ON j.visit_id = v.visit_id WHERE j.symbol LIKE '%Canvas%' ORDER BY v.site_url;"):

        if canvas_font in symbol:
            verif_1.add(symbol + url + val)
            if (sum((canvas_font + url) in s for s in verif_1)) >= 50:
                fontp_1.add(top_url + ' ' + url)

        if canvas_text in symbol:
            verif_2.append(str(vid) + symbol + url + arg)
            if verif_2.count(str(vid) + symbol + url + arg) >= 50:
                fontp_2.add(top_url + ' ' + url)

# Agrupa los sitios que cumplen las dos condiciones del canvas font fingerprinting (Interseccion en un solo set)
    fontp_sites = fontp_1 & fontp_2

    if len(fontp_sites) == 0:
        print('Ninguno de los sitios evaluados aplica esta técnica de fingerprinting')

    else:
# Separamos el set en dos listas
        e, f = zip(*(s.split(' ') for s in fontp_sites))

# Creamos un dataframe con las dos listas
        df3 = pd.DataFrame({'URL Sitio Web': e, 'URL Script': f})
        df3 = df3[['URL Sitio Web', 'URL Script']]

        def make_clickable(val):
            return '<a href="{}">{}</a>'.format(val,val)

#pd.options.display.max_colwidth
        pd.set_option('max_colwidth', -1)

        print('Los sitios web que aplican Fingerprinting de fuentes a través de Canvas según los criterios analizados son los siguientes:')
        display(df3.style.format(make_clickable))

else:
    print('No se ha podido encontrar la base de datos resultante de ejecutar OpenWPM')

In [7]:
button = widgets.Button(button_style='info',description="WebRTC")
button.on_click(run_cell)
display(button)

Button(button_style='info', description='WebRTC', style=ButtonStyle())

In [ ]:
if os.path.isfile("crawl-data.sqlite"):
    
    print("Procesando...")
    clear_output(wait=True)

# Conecta con la base de datos que origina OpenWPM como resultado
    wpm_db = "crawl-data.sqlite"
    conn = lite.connect(wpm_db)
    cur = conn.cursor()

# Define los valores que permiten comprobar las condiciones de webRTC fingerprinting
    rtc_data= "RTCPeerConnection.createDataChannel"
    rtc_offer= "RTCPeerConnection.createOffer"
    rtc_cand= "RTCPeerConnection.onicecandidate"
    rtc_1 = set()
    rtc_2 = set()
    rtc_3 = set()
    rtc_sites_0 = set()
    rtc_sites_1 = set()
    rtc_sites_2 = set()
    rtc_sites_3 = set()
    rtc_sites = set()

# Busca en la base de datos, verificando los sitios donde se cumplen las condiciones de webRTC fingerprinting

    for url, symbol, op, val, arg, top_url in cur.execute("SELECT j.script_url, j.symbol, j.operation, j.value, j.arguments, v.site_url FROM javascript as j JOIN site_visits as v ON j.visit_id = v.visit_id WHERE j.symbol LIKE '%RTCPeerConnection%' ORDER BY v.site_url;"):

        if rtc_data in symbol:
            rtc_1.add(top_url + ' ' + url)

        if rtc_offer in symbol:
            rtc_2.add(top_url + ' ' + url)

        if rtc_cand in symbol:
            rtc_3.add(top_url + ' ' + url)

# Agrupa los sitios que cumplen las condiciones del webRTC fingerprinting (Interseccion en un solo set)
    rtc_sites_0 = rtc_1 & rtc_2 & rtc_3

# Comprueba si la información de webRTC es utilizada en un contexto de fingerprinting

    rtc_sites_1 = rtc_sites_0 & info_sites
    rtc_sites_2 = rtc_sites_0 & fp_sites
    rtc_sites_3 = rtc_sites_0 & fontp_sites

    rtc_sites = rtc_sites_1 | rtc_sites_2 | rtc_sites_3

    if len(rtc_sites) == 0:
        print('Ninguno de los sitios evaluados aplica esta técnica de fingerprinting')

    else:
# Separamos el set en dos listas
        g, h = zip(*(s.split(' ') for s in rtc_sites))

# Creamos un dataframe con las dos listas
        df4 = pd.DataFrame({'URL Sitio Web': g, 'URL Script': h})
        df4 = df4[['URL Sitio Web', 'URL Script']]

        def make_clickable(val):
            return '<a href="{}">{}</a>'.format(val,val)

#pd.options.display.max_colwidth
        pd.set_option('max_colwidth', -1)

        print('Los sitios web que aplican Fingerprinting WebRTC según los criterios analizados son los siguientes:')
        display(df4.style.format(make_clickable))

else:
    print('No se ha podido encontrar la base de datos resultante de ejecutar OpenWPM')

Por defecto se guarda una base de datos donde se acumulan todos los datos obtenidos a través de OpenWPM. Si desea partir en limpio para la siguiente ejecución, presione el botón **Borrar Resultados**.

In [8]:
button = widgets.Button(button_style='danger',description="Borrar Resultados")
button.on_click(run_cell)
display(button)

Button(button_style='danger', description='Borrar Resultados', style=ButtonStyle())

In [ ]:
if os.path.isfile("crawl-data.sqlite"):
    os.remove("crawl-data.sqlite")
if 'info_sites' in locals() or 'info_sites' in globals():
    info_sites.clear()
if 'fp_sites' in locals() or 'fp_sites' in globals():
    fp_sites.clear()
if 'fontp_sites' in locals() or 'fontp_sites' in globals():
    fontp_sites.clear()
if 'rtc_sites' in locals() or 'rtc_sites' in globals():
    rtc_sites.clear()
print("Borrado exitoso. Si desea iniciar en limpio, reinicie el notebook a través de la opción 'Kernel --> Restart & Clear Output'")

<div style="text-align: right"> © 2018 Juan D. Márquez Lagalla </div>